In [1]:
import pandas as pd
import numpy as np

In [2]:
test = pd.read_csv('test_final_100k.csv')

A continuación vamos a realizar las mismas operaciones con los dataframes de postulantes y avisos que realizamos en el notebook principal, a fin de tener un dataframe test con los features correspondientes para que los algoritmos de Machine Learning puedan predecir los resultados.

No realizamos comentarios respecto a las operaciones ya que estos se encuentran en el notebook principal.

## Información de los postulantes

### DF 1: Educación de los postulantes

In [3]:
postulantes_educ = pd.read_csv('datos_navent_fiuba/h15_fiuba_1_postulantes_educacion.csv')
postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, on=['idpostulante', 'nombre', 'estado']\
                       , how='outer')

postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, on=['idpostulante', 'nombre', 'estado']\
                       , how='outer')

postulantes_educ.rename(columns={'nombre':'titulo_univ'}, inplace=True)

postulantes_educ.loc[(postulantes_educ["titulo_univ"] == "Otro"), 'estudios'] = 1
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Secundario'), 'estudios'] = 2
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Terciario/Técnico'), 'estudios'] = 3
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Universitario'), 'estudios'] = 4
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Posgrado'), 'estudios'] = 5
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Master'), 'estudios'] = 6
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Doctorado'), 'estudios'] = 7

postulantes_educ = postulantes_educ.sort_values('estudios', ascending=False)

postulantes_educ = postulantes_educ.drop_duplicates('idpostulante')

postulantes_educ.loc[(postulantes_educ['estado'] == 'Graduado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'Abandonado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'En Curso'), 'esta_estudiando'] = 1

postulantes_educ.drop(columns={'titulo_univ', 'estado'}, axis=1, inplace=True)

postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447909 entries, 393408 to 530578
Data columns (total 3 columns):
idpostulante       447909 non-null object
estudios           447909 non-null float64
esta_estudiando    447909 non-null float64
dtypes: float64(2), object(1)
memory usage: 13.7+ MB


### DF 2: Género y Edad de los postulantes

In [4]:
postulantes_gen_nac = pd.read_csv('datos_navent_fiuba/h15_fiuba_2_postulantes_genero_y_edad.csv')
postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, on=['idpostulante', 'fechanacimiento', 'sexo']\
                       , how='outer')

postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, on=['idpostulante', 'fechanacimiento', 'sexo']\
                       , how='outer')

postulantes_gen_nac = postulantes_gen_nac[postulantes_gen_nac['sexo'] != '0.0']

postulantes_gen_nac = postulantes_gen_nac.sort_values('sexo', ascending=True)

postulantes_gen_nac = postulantes_gen_nac.drop_duplicates('idpostulante')

postulantes_gen_nac = postulantes_gen_nac.rename(columns={'fechanacimiento': 'edad'})

fecha_nac = postulantes_gen_nac['edad'].str.split('-')
año_nac = fecha_nac.str[0]
postulantes_gen_nac['edad'] = año_nac
postulantes_gen_nac['edad'] = pd.to_numeric(postulantes_gen_nac['edad'], errors='coerce').fillna(0).astype(np.int64)
edad_min = postulantes_gen_nac['edad'] > 2000
edad_max = postulantes_gen_nac['edad'] < 1950
condicion_final = ((edad_min | edad_max))
postulantes_gen_nac = postulantes_gen_nac[np.logical_not(condicion_final)]
postulantes_gen_nac['edad'] = postulantes_gen_nac['edad'].apply(lambda x: 2018-x)

rango_18_25 = (postulantes_gen_nac['edad'] >= 18) & (postulantes_gen_nac['edad'] <= 25)
rango_26_30 = (postulantes_gen_nac['edad'] >= 26) & (postulantes_gen_nac['edad'] <= 30)
rango_31_40 = (postulantes_gen_nac['edad'] >= 31) & (postulantes_gen_nac['edad'] <= 40)
rango_41_68 = (postulantes_gen_nac['edad'] >= 41) & (postulantes_gen_nac['edad'] <= 68)
postulantes_gen_nac.loc[(rango_18_25), 'rango_edad'] = 1
postulantes_gen_nac.loc[(rango_26_30), 'rango_edad'] = 2
postulantes_gen_nac.loc[(rango_31_40), 'rango_edad'] = 3
postulantes_gen_nac.loc[(rango_41_68), 'rango_edad'] = 4

postulantes_gen_nac.drop(columns={'edad'}, axis=1, inplace=True)

postulantes_gen_nac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478001 entries, 165770 to 147639
Data columns (total 3 columns):
idpostulante    478001 non-null object
sexo            478001 non-null object
rango_edad      478001 non-null float64
dtypes: float64(1), object(2)
memory usage: 14.6+ MB


### Merge de los DF 1 y 2

In [5]:
postulantes = pd.merge(postulantes_educ, postulantes_gen_nac, on='idpostulante', how='outer')

postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 487459 entries, 0 to 487458
Data columns (total 5 columns):
idpostulante       487459 non-null object
estudios           447909 non-null float64
esta_estudiando    447909 non-null float64
sexo               478001 non-null object
rango_edad         478001 non-null float64
dtypes: float64(3), object(2)
memory usage: 22.3+ MB


In [6]:
postulantes.loc[(postulantes['sexo'] == 'NO_DECLARA'), 'sexo'] = 0
postulantes.loc[(postulantes['sexo'] == 'FEM'),'sexo'] = 1
postulantes.loc[(postulantes['sexo'] == 'MASC'),'sexo'] = 2

postulantes.head()

,idpostulante,estudios,esta_estudiando,sexo,rango_edad
0,1lKjvO,7.0,1.0,NaN,NaN
1,a0XaWD,7.0,0.0,2,4.0
2,4reGo5z,7.0,0.0,1,2.0
3,X9lpKkb,7.0,0.0,2,4.0
4,RzMXJ4E,7.0,0.0,1,4.0


In [7]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 487459 entries, 0 to 487458
Data columns (total 5 columns):
idpostulante       487459 non-null object
estudios           447909 non-null float64
esta_estudiando    447909 non-null float64
sexo               478001 non-null object
rango_edad         478001 non-null float64
dtypes: float64(3), object(2)
memory usage: 22.3+ MB


## Detalles de los avisos

In [8]:
avisos_detalles = pd.read_csv('datos_navent_fiuba/h15_fiuba_6_avisos_detalle.csv')
avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles = avisos_detalles.drop_duplicates(['idaviso'])

avisos_detalles.rename(columns={'denominacion_empresa':'nombre_empresa'}, inplace=True)

avisos_detalles.drop(columns={'idpais', 'ciudad', 'mapacalle'}, axis=1, inplace=True)

avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Gran Buenos Aires'),'nombre_zona'] = 3
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Capital Federal'),'nombre_zona'] = 2
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Buenos Aires (fuera de GBA)'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'GBA Oeste'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'La Plata'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Ciudad de Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Catamarca'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Rosario'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Cruz'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'San Juan'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Cordoba'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Tucuman'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Neuquen'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Fe'),'nombre_zona'] = 1

avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Full-time'),'tipo_de_trabajo'] = 10
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Part-time'),'tipo_de_trabajo'] = 9
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Teletrabajo'),'tipo_de_trabajo'] = 8
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Horas'),'tipo_de_trabajo'] = 7
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Pasantia'),'tipo_de_trabajo'] = 6
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Temporario'),'tipo_de_trabajo'] = 5
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Contrato'),'tipo_de_trabajo'] = 4
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Fines de Semana'),'tipo_de_trabajo'] = 3
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Primer empleo'),'tipo_de_trabajo'] = 2
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Voluntario'),'tipo_de_trabajo'] = 1

avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Gerencia / Alta Gerencia / Dirección'),'nivel_laboral'] = 5
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Jefe / Supervisor / Responsable'),'nivel_laboral'] = 4
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Senior / Semi-Senior'),'nivel_laboral'] = 3
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Junior'),'nivel_laboral'] = 2
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Otro'),'nivel_laboral'] = 1

avisos_detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24950 entries, 0 to 25549
Data columns (total 8 columns):
idaviso            24950 non-null int64
titulo             24950 non-null object
descripcion        24950 non-null object
nombre_zona        24950 non-null int64
tipo_de_trabajo    24950 non-null int64
nivel_laboral      24950 non-null int64
nombre_area        24950 non-null object
nombre_empresa     24943 non-null object
dtypes: int64(4), object(4)
memory usage: 1.7+ MB


## Merge del Test File con la información de los postulantes

In [9]:
test_final = pd.merge(test, postulantes, on='idpostulante', how='left')
test_final.sample(5)

,id,idaviso,idpostulante,estudios,esta_estudiando,sexo,rango_edad
19519,19519,1112228604,EW5QK2,4.0,1.0,2,3.0
47051,47051,1112431002,jkV56Vb,2.0,0.0,2,3.0
4951,4951,1111507445,aGBMGN,3.0,0.0,1,3.0
51827,51827,1112442102,GNObeb9,4.0,0.0,2,2.0
6529,6529,1111651622,aPeDAE,2.0,0.0,2,3.0


In [10]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 7 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
estudios           94388 non-null float64
esta_estudiando    94388 non-null float64
sexo               97019 non-null object
rango_edad         97019 non-null float64
dtypes: float64(3), int64(2), object(2)
memory usage: 6.1+ MB


In [11]:
test_final.loc[(test_final['estudios'].isnull()), 'estudios'] = 0
test_final.loc[(test_final['esta_estudiando'].isnull()), 'esta_estudiando'] = 0
test_final.loc[(test_final['sexo'].isnull()), 'sexo'] = 0
test_final.loc[(test_final['rango_edad'].isnull()), 'rango_edad'] = 0

test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 7 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
estudios           100000 non-null float64
esta_estudiando    100000 non-null float64
sexo               100000 non-null int64
rango_edad         100000 non-null float64
dtypes: float64(3), int64(3), object(1)
memory usage: 6.1+ MB


## Merge del Test Final con los detalles de los avisos

In [12]:
test_final = pd.merge(test_final, avisos_detalles, on='idaviso', how='left')
test_final.sample(5)

,id,idaviso,idpostulante,estudios,esta_estudiando,sexo,rango_edad,titulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nombre_empresa
35914,35914,1112410823,EV0bo,0.0,0.0,0,0.0,Estudiante de Ingeniería,"<p><span style="""">Se busca un estudiante de In...",3.0,10.0,2.0,Ventas,Infinity Rotomolding
30413,30413,1112377689,KBrM66d,2.0,0.0,1,2.0,Vendedores/as,<p>Nos encontramos en búsqueda de candidatos c...,3.0,10.0,3.0,Ventas,Megatlon
92088,92088,1112473722,E8De24,3.0,0.0,1,3.0,Technical Leader - CABA,<p><strong>Stefanini IT Solutions</strong> som...,3.0,10.0,3.0,Tecnologia / Sistemas,Stefanini Argentina SRL
32455,32455,1112393989,eL8pmB,4.0,0.0,1,3.0,OPERADORES TELEFÓNICOS  TELEMARKETERS SIN EXP...,<p>¿Estás en búsqueda trabajo pero no tenes ex...,3.0,2.0,1.0,Ventas,APPLICA - Humanos con Recursos
70496,70496,1112461820,awpRXN,4.0,0.0,2,3.0,Auditor de procesos- Terminal Automotriz Japonesa,<p>Para Terminal Automotriz Japonesa seleccion...,3.0,10.0,3.0,Auditoría,F & C Consultores y Asociados


In [13]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 14 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
estudios           100000 non-null float64
esta_estudiando    100000 non-null float64
sexo               100000 non-null int64
rango_edad         100000 non-null float64
titulo             95552 non-null object
descripcion        95552 non-null object
nombre_zona        95552 non-null float64
tipo_de_trabajo    95552 non-null float64
nivel_laboral      95552 non-null float64
nombre_area        95552 non-null object
nombre_empresa     95540 non-null object
dtypes: float64(6), int64(3), object(5)
memory usage: 11.4+ MB


In [14]:
test_final.loc[(test_final['nombre_zona'].isnull()), 'nombre_zona'] = 0
test_final.loc[(test_final['tipo_de_trabajo'].isnull()), 'tipo_de_trabajo'] = 0
test_final.loc[(test_final['nivel_laboral'].isnull()), 'nivel_laboral'] = 0

test_final.rename(columns={'estudios':'nivel_estudios'}, inplace=True)

test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 14 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
nivel_estudios     100000 non-null float64
esta_estudiando    100000 non-null float64
sexo               100000 non-null int64
rango_edad         100000 non-null float64
titulo             95552 non-null object
descripcion        95552 non-null object
nombre_zona        100000 non-null float64
tipo_de_trabajo    100000 non-null float64
nivel_laboral      100000 non-null float64
nombre_area        95552 non-null object
nombre_empresa     95540 non-null object
dtypes: float64(6), int64(3), object(5)
memory usage: 11.4+ MB


In [15]:
test_final['rango_edad'] = test_final['rango_edad'].astype('category')
test_final['sexo'] = test_final['sexo'].astype('category') 
test_final['nivel_estudios'] = test_final['nivel_estudios'].astype('category')
test_final['esta_estudiando'] = test_final['esta_estudiando'].astype('category')
test_final['tipo_de_trabajo'] = test_final['tipo_de_trabajo'].astype('category')
test_final['nivel_laboral'] = test_final['nivel_laboral'].astype('category')
test_final['nombre_zona'] = test_final['nombre_zona'].astype('category')

test_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 14 columns):
id                 100000 non-null int64
idaviso            100000 non-null int64
idpostulante       100000 non-null object
nivel_estudios     100000 non-null category
esta_estudiando    100000 non-null category
sexo               100000 non-null category
rango_edad         100000 non-null category
titulo             95552 non-null object
descripcion        95552 non-null object
nombre_zona        100000 non-null category
tipo_de_trabajo    100000 non-null category
nivel_laboral      100000 non-null category
nombre_area        95552 non-null object
nombre_empresa     95540 non-null object
dtypes: category(7), int64(2), object(5)
memory usage: 6.8+ MB


## Exportamos a un CSV File el test final

In [16]:
test_final.to_csv('test_final.csv', encoding='utf-8', index=False)